# Note, this *only* works for observing things in the future. Dealing with the pointing corrections in the analysis is not supported here.

# Set your observing times here

In [49]:
from datetime import datetime
tstart = '2017-06-21T00:00:00'
tend = '2017-06-22T00:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

# Step 1, get the nominal RA/Dec Position for Jupiter at the start

In [50]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../data')

ts = load.timescale()
planets = load('de436.bsp')


test_time = (tend - tstart)*0.5 + tstart
print(test_time.isoformat())

astro_time = Time(test_time)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()

print(ra.to(u.deg), dec.to(u.deg))

print(tstart.timetuple().tm_year, tstart.timetuple().tm_yday)
print(tend.timetuple().tm_year, tend.timetuple().tm_yday)


2017-06-21T12:00:00
51.971289477813315 deg 13.604094026059773 deg
2017 172
2017 173


# Step 2: Go run the code that figured out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in ./orbit_engine that use the version that Karl already compiled for the nuops users.

Do:

`./get_latest_TLE.sh`

adjust the

`run_jupiter_test.sh`

script appropriately, then

`./run_jupiter_test.sh`

Check in the resulting jupiter occultation file into the repo, push it here, and them start below.


# Step 3: Parse the resulting occultation file:

In [3]:
file = '../orbit_engine/moon_1_171.occ'
from lunar_planning import *
import lunar_planning
import importlib
importlib.reload(lunar_planning)
orbits = lunar_planning.parse_occ(file)
orbits

,visible,occulted
0,2017-06-18 01:11:22,2017-06-18 02:07:15
1,2017-06-18 02:48:02,2017-06-18 03:43:55
2,2017-06-18 04:24:41,2017-06-18 05:20:35
3,2017-06-18 06:01:21,2017-06-18 06:57:15
4,2017-06-18 07:38:01,2017-06-18 08:33:54
5,2017-06-18 09:14:41,2017-06-18 10:10:34
6,2017-06-18 10:51:20,2017-06-18 11:47:14
7,2017-06-18 12:28:00,2017-06-18 13:23:54
8,2017-06-18 14:04:40,2017-06-18 15:00:33
9,2017-06-18 15:41:20,2017-06-18 16:37:13


# Use SkyField to get the location of the Moon for each orbit:

In [39]:
importlib.reload(lunar_planning)

from datetime import timedelta
buffer = timedelta(minutes=5)



for ind, orbit in orbits.iterrows():
    tstart = orbit['visible'].to_pydatetime() + buffer
    print(tstart, ' to ', tend)
    tend = orbit['occulted'].to_pydatetime() - buffer

    outfile = 'lunar_pointing_'+tstart.isoformat()+'_'+tend.isoformat()
    
    print(outfile)
    
    lunar_planning.get_lunar_radec(tstart, tend,show=True,
                                  parallax_correction=True,
                                  load_path='../data', steps = 5,
                                  outfile=outfile)

    break
    


2017-06-18 01:16:22  to  2017-06-18 02:02:15
lunar_pointing_2017-06-18T01:16:22_2017-06-18T02:02:15
Aim Time            RA                  Dec
Parallax corection (arcsec) 3797.2429388183787
2017-06-18T01:16:22 5.868535158403979  -1.3063805262714114

Parallax corection (arcsec) 3035.9593368560304
2017-06-18T01:25:32.600000 5.7386288496621916  -1.3554448528867307

Parallax corection (arcsec) 1250.0234768039677
2017-06-18T01:34:43.200000 5.309692098711746  -1.3936739438637884

Parallax corection (arcsec) 1171.1907753127878
2017-06-18T01:43:53.800000 4.760868620778142  -1.396861531500884

Parallax corection (arcsec) 2984.274007367352
2017-06-18T01:53:04.400000 4.318956655282572  -1.3538773707828906

